In [109]:
from collections import namedtuple, deque #namedtuple: per definire una struttura semplice e immutabile chiamata Action, che rappresenta una mossa del puzzle.
from heapq import heappush, heappop #heapq: per gestire la coda di priorità (necessaria per algoritmi come A* e Greedy).
import numpy as np #numpy: per manipolare matrici, dato che il puzzle è rappresentato come una griglia 2D.
from random import choice

PUZZLE_DIM = 3
RANDOMIZE_STEPS = 1000

In [110]:
# Define an action as a namedtuple to represent a move in the puzzle
# `pos1` is the position of the empty tile (0), and `pos2` is the tile to swap with.
Action = namedtuple('Action', ['pos1', 'pos2'])

In [111]:
# Class to represent a puzzle state
class State:
    def __init__(self, content: np.ndarray):
        # `content` is the 2D NumPy array representing the current puzzle layout
        self.content = content
        # `hash` provides a unique identifier for this state
        self.hash = hash(content.tobytes())

    # Hashing the state allows us to use it in sets or dictionaries
    def __hash__(self):
        return self.hash

    # Two states are equal if their content is the same
    def __eq__(self, other):
        return np.array_equal(self.content, other.content)

    # Required for priority queues; compares the hash values
    def __lt__(self, other):
        return self.hash < other.hash


In [112]:
# Get all valid actions for the current state
def available_actions(state: np.ndarray) -> list[Action]:
    # Find the position of the empty tile (0)
    x, y = np.argwhere(state == 0)[0]
    actions = []
    # Check possible moves: up, down, left, right
    if x > 0: 
        actions.append(Action((x, y), (x - 1, y)))  # Move up
    if x < state.shape[0] - 1: 
        actions.append(Action((x, y), (x + 1, y)))  # Move down
    if y > 0: 
        actions.append(Action((x, y), (x, y - 1)))  # Move left
    if y < state.shape[1] - 1: 
        actions.append(Action((x, y), (x, y + 1)))  # Move right
    return actions

# Apply an action to the current state and return the resulting new state
def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    # Swap the tiles between `pos1` and `pos2`
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

# Check if a given puzzle state is solvable
def is_solvable(state: np.ndarray) -> bool:
    # Flatten the 2D array into a 1D list to count inversions
    flattened = state.flatten()
    inversions = sum(
        1 for i in range(len(flattened)) for j in range(i + 1, len(flattened))
        if flattened[i] and flattened[j] and flattened[i] > flattened[j]
    )
    # A puzzle is solvable if the number of inversions is even
    return inversions % 2 == 0

In [113]:
# Heuristic: Calculate Manhattan distance for all tiles
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for value in range(1, state.size):  # Exclude the empty tile (0)
        # Find the current and goal positions of the tile
        x1, y1 = np.argwhere(state == value)[0]
        x2, y2 = np.argwhere(goal == value)[0]
        # Add the Manhattan distance (|x1-x2| + |y1-y2|)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Heuristic: Count the number of misplaced tiles
def misplaced_tiles(state: np.ndarray, goal: np.ndarray) -> int:
    # Compare each tile with the goal; subtract 1 to exclude the empty tile (0)
    return np.sum(state != goal) - 1

Breadth-First Search (BFS) algorithm

In [114]:
# Breadth-First Search (BFS) algorithm
def bfs(start: np.ndarray, goal: np.ndarray) -> list[Action]:
    queue = [(State(start), [])]  # Queue contains states and their action paths
    visited = set()  # To track visited states
    while queue:
        current, path = queue.pop(0)  # Explore the oldest state in the queue
        if np.array_equal(current.content, goal):  # Check if goal is reached
            return path
        visited.add(current)  # Mark the state as visited
        for action in available_actions(current.content):
            neighbor = State(do_action(current.content, action))
            if neighbor not in visited:
                queue.append((neighbor, path + [action]))
    return None

A* Search Algorithm

In [115]:
# A* Search algorithm
def astar(start: np.ndarray, goal: np.ndarray, heuristic) -> list[Action]:
    open_set = []
    heappush(open_set, (0, State(start), []))  # Priority queue with (cost, state, path)
    visited = set()
    while open_set:
        _, current, path = heappop(open_set)  # Extract the state with the lowest cost
        if np.array_equal(current.content, goal):  # Check if goal is reached
            return path
        visited.add(current)  # Mark the state as visited
        for action in available_actions(current.content):
            neighbor = State(do_action(current.content, action))
            if neighbor not in visited:
                # Calculate total cost = path cost + heuristic cost
                cost = len(path) + 1 + heuristic(neighbor.content, goal)
                heappush(open_set, (cost, neighbor, path + [action]))
    return None

Greedy Best-First Search algorithm

In [116]:
# Greedy Best-First Search algorithm
def greedy(start: np.ndarray, goal: np.ndarray, heuristic) -> list[Action]:
    open_set = []
    heappush(open_set, (0, State(start), []))  # Priority queue with (heuristic cost, state, path)
    visited = set()
    while open_set:
        _, current, path = heappop(open_set)  # Extract the state with the lowest heuristic cost
        if np.array_equal(current.content, goal):  # Check if goal is reached
            return path
        visited.add(current)  # Mark the state as visited
        for action in available_actions(current.content):
            neighbor = State(do_action(current.content, action))
            if neighbor not in visited:
                # Use heuristic cost only (ignores path cost)
                cost = heuristic(neighbor.content, goal)
                heappush(open_set, (cost, neighbor, path + [action]))
    return None

In [117]:
# Function to generate a random initial state by shuffling the goal state
def init_state(goal: np.ndarray) -> np.ndarray:
    state = goal.copy()
    for _ in range(RANDOMIZE_STEPS):
        state = do_action(state, choice(available_actions(state)))
    return state


def solve_puzzle():
    goal = np.arange(1, PUZZLE_DIM**2 + 1).reshape((PUZZLE_DIM, PUZZLE_DIM))
    goal[-1, -1] = 0
    state = init_state(goal)
    
    if not is_solvable(state):
        print("Puzzle non risolvibile!")
        return
    
    print("Stato iniziale:")
    print(state)
    print("\nSolution with BFS:")
    bfs_solution = bfs(state, goal)
    print(f"Number of moves: {len(bfs_solution)}")

    print("\nSolution with A* (Manhattan Distance):")
    astar_solution = astar(state, goal, manhattan_distance)
    print(f"Number of moves: {len(astar_solution)}")

    print("\nSolution with Greedy (Misplaced Tiles):")
    greedy_solution = greedy(state, goal, misplaced_tiles)
    print(f"Number of moves: {len(greedy_solution)}")


# Execute the program
solve_puzzle()

Stato iniziale:
[[4 7 1]
 [5 0 2]
 [8 3 6]]

Solution with BFS:
Number of moves: 18

Solution with A* (Manhattan Distance):
Number of moves: 18

Solution with Greedy (Misplaced Tiles):
Number of moves: 60
